In [107]:
import pandas as pd
import re

In [4]:
knet = pd.read_json("/datasets/KNET/train.json")
knet.head()

,fold,source,documentId,documentText,passages
0,3,Diffbot Knowledge Graph,13205,Documents Assist\n\nFlorida Cheap Divorce - Si...,"[{'passageId': '13205:453:779', 'exhaustivelyA..."
1,2,Diffbot Knowledge Graph,7,Tata Chemicals Europe\n\nTata Chemicals Europe...,"[{'passageId': '7:23:206', 'exhaustivelyAnnota..."
2,1,Diffbot Knowledge Graph,29726,Morphine\n\nMorphine was an American alternati...,"[{'passageId': '29726:151:315', 'exhaustivelyA..."
3,4,Diffbot Knowledge Graph,13115,Norman Nadorff\n\nNorman J. Nadorff is an inte...,"[{'passageId': '13115:1833:1922', 'exhaustivel..."
4,2,DBpedia Abstract,7109,Abdullah bin Mohammed bin Saud Al Thani\n\nH.E...,"[{'passageId': '7109:41:205', 'exhaustivelyAnn..."


In [10]:
print(knet.iloc[0].documentText)

Documents Assist

Florida Cheap Divorce - Simple Divorce $280 flat fee - No Court Divorce
Established in 2006.
After a few years working as an Independent Associate of Legal Shield, formerly known as Pre-Paid Legal, Yamil Fuentes made it her mission to help her clients take advantage of their access to the legal system. After going through her own personal divorce she realized there was a need of affordable divorce services in the State of Florida.
When she founded Apex Legal Document Preparation Services she became a Premium Member of the FALDP (Florida Association of Legal Document Preparers) and educated herself on the different types of divorce been offered across the State of Florida and learned from the best in the industry of legal document preparation services.
Through her membership she embarked on a journey to learn how to make the divorce process as easy as possible for her clients, making her company the most sought after divorce preparer in Florida touting the following se

In [33]:
knet[knet.passages.apply(len) > 2].iloc[0].passages
# .passages[0]["facts"]

[{'passageId': '1756:399:488',
  'exhaustivelyAnnotatedProperties': [{'propertyId': '5',
    'propertyName': 'DATE_FOUNDED',
    'propertyDescription': 'Describes the founding date of a company.'}],
  'passageStart': 399,
  'passageEnd': 488,
  'passageText': 'His passion was coin collecting and he also co-founded the South Shore Coin Club in 1972.',
  'facts': [{'factId': '1756:458:479:483:487:5',
    'propertyId': '5',
    'humanReadable': '<South Shore Coin Club> <DATE_FOUNDED> <1972>',
    'annotatedPassage': 'His passion was coin collecting and he also co-founded the <South Shore Coin Club> in <1972>.',
    'subjectStart': 458,
    'subjectEnd': 479,
    'subjectText': 'South Shore Coin Club',
    'subjectUri': '',
    'objectStart': 483,
    'objectEnd': 487,
    'objectText': '1972',
    'objectUri': ''}]},
 {'passageId': '1756:299:398',
  'exhaustivelyAnnotatedProperties': [{'propertyId': '3',
    'propertyName': 'EMPLOYEE_OR_MEMBER_OF',
    'propertyDescription': 'Describes th

## Relations

In [56]:
relations_df = knet.passages.apply(lambda t: list(map(lambda x: x["exhaustivelyAnnotatedProperties"], t)))
relations_df

0       [[{'propertyId': '9', 'propertyName': 'EDUCATE...
1       [[{'propertyId': '1', 'propertyName': 'SUBSIDI...
2       [[{'propertyId': '14', 'propertyName': 'DATE_O...
3       [[{'propertyId': '10', 'propertyName': 'NATION...
4       [[{'propertyId': '10', 'propertyName': 'NATION...
                              ...                        
3972    [[{'propertyId': '45', 'propertyName': 'POLITI...
3973    [[{'propertyId': '10', 'propertyName': 'NATION...
3974    [[{'propertyId': '1', 'propertyName': 'SUBSIDI...
3975    [[{'propertyId': '14', 'propertyName': 'DATE_O...
3976    [[{'propertyId': '5', 'propertyName': 'DATE_FO...
Name: passages, Length: 3977, dtype: object

In [62]:
relations = relations_df.apply(lambda r: list(map(lambda x: x[0]["propertyName"], r)))

In [63]:
relations = set(sum(relations, []))
relations

{'CEO',
 'CHILD_OF',
 'DATE_FOUNDED',
 'DATE_OF_BIRTH',
 'DATE_OF_DEATH',
 'EDUCATED_AT',
 'EMPLOYEE_OR_MEMBER_OF',
 'FOUNDED_BY',
 'HEADQUARTERS',
 'NATIONALITY',
 'PLACE_OF_BIRTH',
 'PLACE_OF_RESIDENCE',
 'POLITICAL_AFFILIATION',
 'SPOUSE',
 'SUBSIDIARY_OF'}

## Eval data

In [67]:
import gc
import torch
from mistral_inference.transformer import Transformer
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

from mistral_inference.generate import generate
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

In [76]:
model = None
tokenizer = None


def load_model():
    global model, tokenizer
    tokenizer = MistralTokenizer.from_file("/models/M7B/tokenizer.model.v3")  # change to extracted tokenizer file
    model = Transformer.from_folder("/models/M7B")  # change to extracted model dir



def clear_model():
    global model, tokenizer
    del model, tokenizer
    gc.collect()
    torch.cuda.empty_cache()


def get_response(prompt):
    global model, tokenizer
    completion_request = ChatCompletionRequest(messages=[UserMessage(content=prompt)])
    tokens = tokenizer.encode_chat_completion(completion_request).tokens
    out_tokens, _ = generate([tokens], model, max_tokens=1024, temperature=0.35, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
    result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])
    return result

In [70]:
eval_knet = knet[knet.fold == 4]

In [79]:
print(eval_knet.iloc[0].documentText)

Norman Nadorff

Norman J. Nadorff is an international energy attorney with more than 30 years of experience working for the law departments of major oil and gas companies. He retired from BP in 2015 after 26 years (including 11 years with ARCO), the last nine of which were in BP Angola. Mr. Nadorff has done E&P legal work in Latin America, Southeast Asia West Africa and elsewhere for Conoco, ARCO and BP, including two expatriate assignments in Indonesia. At ARCO, he wrote the Company's policies on Foreign Corrupt Practices Act, U.S. Anti-boycott laws and U.S. Export Regulations.
Mr. Nadorff is also an Adjunct Professor of Law at the University of Houston Law Center and a frequent international lecturer. He has given law classes at University of Texas, University of Miami, Agostinho Neto University (Angola), Chulalongkorn University (Bangkok) and ITAM (Mexico City).
Mr. Nadorff is a former Director-at-Large of the Association of International Petroleum Negotiators ("AIPN") and a present

In [73]:
RELATION_EXTRACTION_PROMPT = """
Below you will find a passage text, your task
is to extract the entity relation triplets
in the following format: <Subject>:<Predicate>:<Object>
You should extract and number all possible triplets in a given passage.
PASSAGE:
{passage}
"""

In [92]:
load_model()

In [106]:
clear_model()

In [91]:
passage = eval_knet.iloc[1]
passage.passages

[{'passageId': '4927:374:480',
  'exhaustivelyAnnotatedProperties': [{'propertyId': '2',
    'propertyName': 'FOUNDED_BY',
    'propertyDescription': 'Describes the relationship between an organization and its founder or co-founder.'},
   {'propertyId': '3',
    'propertyName': 'EMPLOYEE_OR_MEMBER_OF',
    'propertyDescription': 'Describes the previous/current employer of a person.'},
   {'propertyId': '4',
    'propertyName': 'CEO',
    'propertyDescription': 'Describes the relationship between an organization and its CEO.'},
   {'propertyId': '5',
    'propertyName': 'DATE_FOUNDED',
    'propertyDescription': 'Describes the founding date of a company.'}],
  'passageStart': 374,
  'passageEnd': 480,
  'passageText': 'Of Armenian descent, he was the founder and first president of the Armenian Academy of Sciences (1943–47).',
  'facts': [{'factId': '4927:395:397:441:469:3',
    'propertyId': '3',
    'humanReadable': '<he> <EMPLOYEE_OR_MEMBER_OF> <Armenian Academy of Sciences>',
    'an

In [94]:
output = get_response(RELATION_EXTRACTION_PROMPT.format(passage=passage.documentText))

In [ ]:
print(output) 

1. <Joseph Orbeli>:<was born on>:<March 20 (O.S. 8 March) 1887>
2. <Joseph Orbeli>:<died on>:<February 2, 1961>
3. <Joseph Orbeli>:<was a>:<renowned Soviet orientalist>
4. <Joseph Orbeli>:<specialized in>:<medieval history of Southern Caucasus>
5. <Joseph Orbeli>:<administered>:<the State Hermitage Museum in Leningrad from 1934 to 1951>
6. <Joseph Orbeli>:<was of>:<Armenian descent>
7. <Joseph Orbeli>:<was the founder and first president of>:<the Armenian Academy of Sciences (1943–47)>
8. <Joseph Orbeli (Armenian name)>:<is>:<Հովսեփ Աբգարի Օրբելի (Hovsep Abgari Orbeli)>
9. <Joseph Orbeli (Russian name)>:<is>:<Иосиф Абгарович Орбели (Iosif Abgarovich Orbeli)>


In [123]:
def extract_triplet(text):
    pattern = re.compile("\d. (.*),?")
    matches = re.findall(pattern, text)
    return matches[0].split(":")

In [124]:
list(map(extract_triplet, output.split("\n")))

[['<Joseph Orbeli>', '<was born on>', '<March 20 (O.S. 8 March) 1887>'],
 ['<Joseph Orbeli>', '<died on>', '<February 2, 1961>'],
 ['<Joseph Orbeli>', '<was a>', '<renowned Soviet orientalist>'],
 ['<Joseph Orbeli>',
  '<specialized in>',
  '<medieval history of Southern Caucasus>'],
 ['<Joseph Orbeli>',
  '<administered>',
  '<the State Hermitage Museum in Leningrad from 1934 to 1951>'],
 ['<Joseph Orbeli>', '<was of>', '<Armenian descent>'],
 ['<Joseph Orbeli>',
  '<was the founder and first president of>',
  '<the Armenian Academy of Sciences (1943–47)>'],
 ['<Joseph Orbeli (Armenian name)>',
  '<is>',
  '<Հովսեփ Աբգարի Օրբելի (Hovsep Abgari Orbeli)>'],
 ['<Joseph Orbeli (Russian name)>',
  '<is>',
  '<Иосиф Абгарович Орбели (Iosif Abgarovich Orbeli)>']]

In [105]:
list(map(lambda t: t["humanReadable"], passage.passages[0]["facts"]))

['<he> <EMPLOYEE_OR_MEMBER_OF> <Armenian Academy of Sciences>',
 '<Armenian Academy of Sciences> <DATE_FOUNDED> <1943>',
 '<Armenian Academy of Sciences> <FOUNDED_BY> <he>']